# <center> Introduction to Spark In-memmory Computing via Python PySpark </center>

In [1]:
!module list

Currently Loaded Modulefiles:
  1) anaconda3/4.2.0   3) zeromq/4.1.5
  2) matlab/2015a      4) hdp/0.1


In [2]:
import sys
import os

sys.path.insert(0, '/usr/hdp/2.6.0.3-8/spark2/python')
sys.path.insert(0, '/usr/hdp/2.6.0.3-8/spark2/python/lib/py4j-0.10.4-src.zip')

os.environ['SPARK_HOME'] = '/usr/hdp/2.6.0.3-8/spark2/'
os.environ['SPARK_CONF_DIR'] = '/etc/hadoop/synced_conf/spark2/'
os.environ['PYSPARK_PYTHON'] = '/software/anaconda3/4.2.0/bin/python'

import pyspark
conf = pyspark.SparkConf()
conf.setMaster("yarn")
conf.set("spark.driver.memory","4g")
conf.set("spark.executor.memory","60g")
conf.set("spark.num.executors","3")
conf.set("spark.executor.cores","12")

sc = pyspark.SparkContext(conf=conf)

In [3]:
sc

### Movie Ratings

An independent movie company is looking to invest in a new movie project. With limited finance, the company wants to 
analyze the reaction of audiences, particularly toward various movie genres, in order to identify beneficial 
movie project to focus on. The company relies on data collected from a publicly available recommendation service 
by [MovieLens](http://dl.acm.org/citation.cfm?id=2827872). This 
[dataset](http://files.grouplens.org/datasets/movielens/ml-10m-README.html) contains **24404096** ratings and **668953**
 tag applications across **40110** movies. These data were created by **247753** users between January 09, 1995 and January 29, 2016. This dataset was generated on October 17, 2016. 

From this dataset, several analyses are possible, include the followings:
1.   Find movies which have the highest average ratings over the years and identify the corresponding genre.
2.   Find genres which have the highest average ratings over the years.
3.   Find users who rate movies most frequently in order to contact them for in-depth marketing analysis.

These types of analyses, which are somewhat ambiguous, demand the ability to quickly process large amount of data in 
elatively short amount of time for decision support purposes. In these situations, the sizes of the data typically 
make analysis done on a single machine impossible and analysis done using a remote storage system impractical. For 
remainder of the lessons, we will learn how HDFS provides the basis to store massive amount of data and to enable 
the programming approach to analyze these data.

In [ ]:
!hdfs dfs -ls /repository/movielens

In [ ]:
!hdfs dfs -cat  /repository/movielens/README.txt

In [ ]:
!hdfs dfs -cat  /repository/movielens/links.csv \
    2>/dev/null | head -n 5

In [ ]:
!hdfs dfs -cat  /repository/movielens/movies.csv \
    2>/dev/null | head -n 5

In [ ]:
!hdfs dfs -cat  /repository/movielens/ratings.csv \
    2>/dev/null | head -n 5

In [ ]:
!hdfs dfs -cat  /repository/movielens/tags.csv \
    2>/dev/null | head -n 5

In [3]:
ratings = sc.textFile("/repository/movielens/ratings.csv")

In [4]:
ratings.cache()

/repository/movielens/ratings.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [5]:
%%time
ratings.count()

CPU times: user 18.2 ms, sys: 3.41 ms, total: 21.6 ms
Wall time: 14 s


24404097

In [6]:
%%time
ratings.count()

CPU times: user 11 ms, sys: 3.93 ms, total: 14.9 ms
Wall time: 8.33 s


24404097

In [7]:
%%time
ratings.count()

CPU times: user 11.6 ms, sys: 3.27 ms, total: 14.8 ms
Wall time: 8.88 s


24404097

### 4.1 Find movies which have the highest average ratings over the years and identify the corresponding genre

- Find the average ratings of all movies over the years
- Identify the corresponding genres for each movie

In [8]:
ratings.take(5)

['userId,movieId,rating,timestamp',
 '1,122,2.0,945544824',
 '1,172,1.0,945544871',
 '1,1221,5.0,945544788',
 '1,1441,4.0,945544871']

In [9]:
ratingHeader = ratings.first() #extract header
print(ratingHeader)

userId,movieId,rating,timestamp


In [10]:
ratingsOnly = ratings.filter(lambda x:x != ratingHeader)

In [11]:
ratingsOnly.take(5)

['1,122,2.0,945544824',
 '1,172,1.0,945544871',
 '1,1221,5.0,945544788',
 '1,1441,4.0,945544871',
 '1,1609,3.0,945544824']

In [12]:
movieRatings = ratingsOnly.map(lambda line: (line.split(",")[1], float(line.split(",")[2])))

In [13]:
movieRatings.take(5)

[('122', 2.0), ('172', 1.0), ('1221', 5.0), ('1441', 4.0), ('1609', 3.0)]

**Possible approaches in aggregating data:** 
- groupByKey and mapValues
- reduceByKey and countByKey

**groupByKey and mapValues**

In [14]:
groupByKeyRatings = movieRatings.groupByKey()

groupByKeyRatings.take(5)

[('154214', <pyspark.resultiterable.ResultIterable at 0x2b508e278908>),
 ('81548', <pyspark.resultiterable.ResultIterable at 0x2b508e278978>),
 ('53161', <pyspark.resultiterable.ResultIterable at 0x2b508e278a58>),
 ('118702', <pyspark.resultiterable.ResultIterable at 0x2b508e2789e8>),
 ('109269', <pyspark.resultiterable.ResultIterable at 0x2b508e278b38>)]

In [15]:
mapValuesToListRatings = groupByKeyRatings.mapValues(list)
mapValuesToListRatings.take(5)

[('154214', [0.5, 3.5, 2.0]),
 ('81548', [4.0, 3.0, 2.0]),
 ('7505',
  [4.5,
   5.0,
   4.5,
   5.0,
   4.5,
   3.0,
   5.0,
   3.5,
   2.5,
   4.5,
   3.5,
   4.0,
   4.0,
   5.0,
   5.0,
   4.5,
   5.0,
   4.0,
   4.5,
   5.0,
   4.0,
   4.5,
   4.5,
   3.5,
   4.0,
   4.5,
   4.5,
   4.0,
   4.0,
   4.5,
   5.0,
   3.5,
   2.5,
   5.0,
   4.0,
   1.5,
   4.0,
   5.0,
   2.5,
   5.0,
   2.0,
   2.5,
   4.0,
   4.5,
   4.5,
   3.5,
   5.0,
   4.5,
   4.5,
   4.5,
   4.0,
   5.0,
   4.0,
   4.0,
   3.5,
   3.5,
   4.0,
   4.0,
   4.0,
   4.0,
   0.5,
   1.5,
   3.5,
   5.0,
   4.0,
   4.0,
   4.0,
   4.0,
   3.0,
   4.5,
   4.5,
   4.5,
   3.0,
   4.5,
   5.0,
   3.5,
   2.5,
   4.0,
   5.0,
   3.0,
   4.0,
   5.0,
   2.0,
   3.5,
   4.0,
   4.0,
   5.0,
   5.0,
   4.5,
   4.0,
   5.0,
   4.0,
   3.5,
   4.5,
   3.5,
   4.0,
   4.0,
   2.5,
   3.5,
   3.5,
   3.5,
   0.5,
   4.5,
   4.0,
   4.0,
   3.5,
   4.0,
   4.5,
   4.0,
   4.0,
   5.0,
   4.0,
   4.0,
   4.0,
   5.0,
   4.0,
   

In [16]:
avgRatings01 = mapValuesToListRatings.mapValues(lambda V: sum(V) / float(len(V)))

avgRatings01.take(5)

[('154214', 2.0),
 ('81548', 3.0),
 ('53161', 3.646892655367232),
 ('118702', 3.538922155688623),
 ('109269', 2.1666666666666665)]

Is this correct?

In [17]:
(3.5 + 3.5 + 2.5 + 3.5 + 2.0 + 3.5 + 2.5 + 3.0) / 8

3.0

**reduceByKey and countByKey**

In [18]:
countsByKey = movieRatings.countByKey()

countsByKey

defaultdict(int,
            {'79855': 5,
             '51891': 5,
             '3981': 4002,
             '125': 3633,
             '643': 28,
             '106897': 5,
             '142376': 2,
             '6998': 155,
             '113836': 1,
             '111': 29909,
             '112138': 1909,
             '115929': 9,
             '152220': 2,
             '32866': 14,
             '127007': 3,
             '158294': 2,
             '78465': 6,
             '107769': 111,
             '4077': 751,
             '160850': 1,
             '141315': 2,
             '2190': 345,
             '90206': 7,
             '125397': 1,
             '131427': 1,
             '4846': 1411,
             '33823': 47,
             '45652': 10,
             '39412': 4,
             '106310': 3,
             '83777': 11,
             '1237': 4945,
             '6795': 96,
             '51088': 431,
             '159462': 7,
             '122199': 1,
             '143128': 1,
             '16207

In [19]:
def sumValues(x,y):
    return (x + y)

sumRatings = movieRatings.reduceByKey(sumValues)

sumRatings.take(5)

[('154214', 6.0),
 ('81548', 9.0),
 ('7505', 1580.5),
 ('109269', 6.5),
 ('3665', 323.0)]

In [20]:
import operator

sumRatings = movieRatings.reduceByKey(operator.add)
sumRatings.take(5)

[('154214', 6.0),
 ('81548', 9.0),
 ('53161', 1291.0),
 ('118702', 1773.0),
 ('109269', 6.5)]

In [21]:
avgRatings02 = sumRatings.map(lambda x: (x[0], x[1] / countsByKey.get(x[0])))

avgRatings02.take(5)

[('154214', 2.0),
 ('81548', 3.0),
 ('53161', 3.646892655367232),
 ('118702', 3.538922155688623),
 ('109269', 2.1666666666666665)]

How do we augment movie ratings data with title informations?

In [22]:
movies = sc.textFile("/repository/movielens/movies.csv")

In [23]:
movieHeader = movies.first() #extract header
print(movieHeader)

movieId,title,genres


In [24]:
movies = movies.filter(lambda x:x != movieHeader)

movies.take(5)

['1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy',
 '2,Jumanji (1995),Adventure|Children|Fantasy',
 '3,Grumpier Old Men (1995),Comedy|Romance',
 '4,Waiting to Exhale (1995),Comedy|Drama|Romance',
 '5,Father of the Bride Part II (1995),Comedy']

In [25]:
movieInfo = movies.map(lambda line: (line.split(",")[0], (line.split(",")[1], line.split(",")[2])))

movieInfo.take(5)

[('1', ('Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy')),
 ('2', ('Jumanji (1995)', 'Adventure|Children|Fantasy')),
 ('3', ('Grumpier Old Men (1995)', 'Comedy|Romance')),
 ('4', ('Waiting to Exhale (1995)', 'Comedy|Drama|Romance')),
 ('5', ('Father of the Bride Part II (1995)', 'Comedy'))]

In [26]:
augmentedRatings = avgRatings01.join(movieInfo)

augmentedRatings.take(5)

[('38061',
  (3.855664488017429,
   ('Kiss Kiss Bang Bang (2005)', 'Comedy|Crime|Mystery|Thriller'))),
 ('59154',
  (2.9615384615384617, ('Black Friday (1940)', 'Crime|Drama|Horror|Sci-Fi'))),
 ('3665',
  (1.8563218390804597,
   ('Curse of the Puppet Master (Puppet Master 6: The Curse) (1998)',
    'Horror|Sci-Fi|Thriller'))),
 ('4936', (3.2678837555886737, ('Fame (1980)', 'Drama|Musical'))),
 ('109769',
  (3.4634146341463414,
   ('"Keeper of Lost Causes', ' The (Kvinden i buret) (2013)"')))]

*Movie with highest average rating:*

In [27]:
augmentedRatings.takeOrdered(10, key = lambda x : -x[1][0])

[('140379',
  (5.0, ('The Face on the Milk Carton (1995)', '(no genres listed)'))),
 ('157589', (5.0, ('Villan (2002)', 'Action|Crime'))),
 ('141100', (5.0, ('Sujata (1960)', 'Drama|Romance'))),
 ('158958', (5.0, ('Pollyanna (2003)', 'Children|Drama'))),
 ('128003', (5.0, ('Fantômas: Juve Against Fantômas (1913)', 'Crime|Drama'))),
 ('140333', (5.0, ('Maundy Thursday (2006)', 'Drama'))),
 ('151619', (5.0, ('Strangers (2007)', 'Drama|Romance|Thriller'))),
 ('136826', (5.0, ('The Defiant Ones (1986)', 'Drama'))),
 ('140369',
  (5.0, ('War Arrow (1954)', 'Adventure|Drama|Romance|War|Western'))),
 ('155709', (5.0, ('Bad Spelling (2004)', 'Drama')))]

*Movie with lowest average rating:*

In [28]:
augmentedRatings.takeOrdered(10, key = lambda x : x[1][0])

[('138070', (0.5, ('Deserter (2002)', 'Drama|War'))),
 ('102459', (0.5, ('Andrew Dice Clay: Indestructible (2012)', 'Comedy'))),
 ('124322',
  (0.5, ('House Party 4: Down to the Last Minute (2001)', 'Comedy'))),
 ('129875', (0.5, ('Pigs with a P.38 (1978)', '(no genres listed)'))),
 ('110800', (0.5, ('"Second Man', ' The (O Defteros Andras) (2013)"'))),
 ('162580', (0.5, ('Paradise (2015)', '(no genres listed)'))),
 ('141088', (0.5, ('La pecora nera (1968)', 'Comedy'))),
 ('123202', (0.5, ('Social Nightmare (2013)', 'Drama|Mystery'))),
 ('105481', (0.5, ('Blood Shack (1971)', 'Horror'))),
 ('158671', (0.5, ('CyberTracker (1994)', 'Action|Sci-Fi')))]

### Challenge

- Augment the mapping process of WordCount with a function to filter out punctuations and capitalization from the unique words

In [29]:
textFile = sc.textFile("/repository/gutenberg-shakespeare.txt")

def elaborateUpper(x):
    return x.upper()

wordcount = textFile.flatMap(lambda line: line.split(" ")) \
            .map(lambda word: (elaborateUpper(word), 1)) \
            .reduceByKey(lambda a, b: a + b)
        
wordcount.take(20)

[('', 516839),
 ('PAGE;', 18),
 ('WITNESSING', 2),
 ('LIBRARY,', 219),
 ('CONFEDERATES,', 4),
 ("COMPASS'D?", 1),
 ("GOODS.'", 1),
 ("'HORUM';", 1),
 ('STUMBLE', 5),
 ('SEYTON!', 1),
 ('RAMSTON,', 1),
 ('SMUG', 3),
 ('EXECUTIONER;', 1),
 ("SMELL'D", 1),
 ('[BEROWNE', 3),
 ('SELF-SAME', 17),
 ('ASKED,', 1),
 ('[SHEATHES', 2),
 ('DOUBT!', 1),
 ('WELL:', 11)]

### Challenge:

1. Make appropriate changes so that only movies with averaged ratings higher than 3.75 are collected
2. Further enhance your modification so that only movies with averaged ratings higher than 3.75 and number of ratings of at least 1000 times are collected.

### 4.2 Find genres which have the highest average ratings over the years

- Identify the genres associated with a movie and its rating
- Each movie can have multiple genres. How to flip the Key/Value pair?

In [29]:
movieRatings.take(5)

[('122', 2.0), ('172', 1.0), ('1221', 5.0), ('1441', 4.0), ('1609', 3.0)]

In [30]:
movieInfo.take(5)

[('1', ('Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy')),
 ('2', ('Jumanji (1995)', 'Adventure|Children|Fantasy')),
 ('3', ('Grumpier Old Men (1995)', 'Comedy|Romance')),
 ('4', ('Waiting to Exhale (1995)', 'Comedy|Drama|Romance')),
 ('5', ('Father of the Bride Part II (1995)', 'Comedy'))]

In [31]:
augmentedInfo = movieRatings.join(movieInfo)

In [32]:
augmentedInfo.take(5)

[('38061',
  (4.0, ('Kiss Kiss Bang Bang (2005)', 'Comedy|Crime|Mystery|Thriller'))),
 ('38061',
  (3.5, ('Kiss Kiss Bang Bang (2005)', 'Comedy|Crime|Mystery|Thriller'))),
 ('38061',
  (3.0, ('Kiss Kiss Bang Bang (2005)', 'Comedy|Crime|Mystery|Thriller'))),
 ('38061',
  (5.0, ('Kiss Kiss Bang Bang (2005)', 'Comedy|Crime|Mystery|Thriller'))),
 ('38061',
  (5.0, ('Kiss Kiss Bang Bang (2005)', 'Comedy|Crime|Mystery|Thriller')))]

In [44]:
def extractGenreRating (t):
    final_tuples = []
    genreList = t[1][1][1].split("|")
    for genre in genreList:
        final_tuples.append((genre,t[1][0]))
    return final_tuples

print(extractGenreRating((u'1', (3.0, (u'Toy Story (1995)', u'Adventure|Animation|Children|Comedy|Fantasy')))))

[('Adventure', 3.0), ('Animation', 3.0), ('Children', 3.0), ('Comedy', 3.0), ('Fantasy', 3.0)]


In [ ]:
genreRatings = augmentedInfo.flatMap(extractGenreRating)

In [43]:
countsByKey = genreRatings.countByKey()

countsByKey

defaultdict(int,
            {' The (Innocente': 7,
             ' The (Signe du lion': 8,
             ' A (a.k.a. A Winter\'s Tale) (Conte d\'hiver) (1992)"': 17,
             ' The (2000)"': 121569,
             ' The (Uchû daikaijû Girara) (1967)"': 4,
             ' The (Giardino dei Finzi-Contini': 1224,
             ' The (El secreto de sus ojos) (2009)"': 1923,
             ' Younger Sister (Ani imôto) (1953)"': 2,
             ' Violent': 1,
             ' The (Sampo) (1959)"': 25,
             ' The (Follow Me!) (1972)"': 2,
             ' The (Painija) (1985)"': 8,
             ' The (Vikaren) (2007)"': 7,
             ' Happy (Sykt lykkelig) (2010)"': 17,
             ' The (Pas si méchant que ça) (1975)"': 2,
             ' White Cat (Crna macka': 1651,
             ' the Flesh and the Devil': 9,
             ' A (Mercenary': 15,
             ' Totally (2006)"': 54,
             ' The (Imbalsamatore': 20,
             ' The (1941)"': 16928,
             ' The (A Londoni fé

In [35]:
genreRatings.take(5)

[('Comedy', 4.5),
 ('Crime', 4.5),
 ('Mystery', 4.5),
 ('Thriller', 4.5),
 ('Comedy', 2.5)]

### Challenge:

Complete the remaining portion of task 2.2: Calculating the average rating of each genre over the years

In [ ]:
groupByKey/mapValues

### 4.3 Find users who rate movies most frequently in order to contact them for in-depth marketing analysis

- How do you define "frequently"?
    - At least once per week?

In [36]:
userRatings = ratingsOnly.map(lambda line: (line.split(",")[0], float(line.split(",")[3])))

In [37]:
ratingGroupByUsers = userRatings.groupByKey().mapValues(list)
ratingGroupByUsers.take(5)

[('231424',
  [831912501.0,
   831912625.0,
   831912398.0,
   831912501.0,
   831912464.0,
   831912542.0,
   831912398.0,
   831912439.0,
   831912439.0,
   831912398.0,
   831912439.0,
   831912398.0,
   831912398.0]),
 ('197183',
  [865885051.0,
   865885143.0,
   865885053.0,
   865885104.0,
   865885052.0,
   865885104.0,
   865885104.0,
   865885236.0,
   865885051.0,
   865885209.0,
   865885143.0,
   865885181.0,
   865885050.0,
   865885048.0,
   865885104.0,
   865885236.0,
   865885162.0]),
 ('41904',
  [1241843329.0,
   1241843268.0,
   1241843302.0,
   1241843244.0,
   1241843292.0,
   1241843276.0,
   1241843251.0,
   1241843313.0,
   1241843263.0,
   1241843333.0,
   1241843233.0,
   1241843240.0,
   1241843282.0,
   1241843265.0]),
 ('81548',
  [1469644107.0,
   1469644062.0,
   1469644206.0,
   1469644041.0,
   1469643981.0,
   1469644179.0,
   1469644145.0,
   1469644141.0,
   1469644111.0,
   1469644130.0,
   1469644174.0]),
 ('223127',
  [1471894540.0,
   147189435

In [38]:
avgRatingFreq = ratingGroupByUsers.mapValues(lambda V: (max(V) - min(V)) / float(len(V)))
avgRatingFreq.take(5)

[('231424', 17.46153846153846),
 ('197183', 11.058823529411764),
 ('41904', 7.142857142857143),
 ('81548', 20.454545454545453),
 ('223127', 15089.424489795918)]

In [39]:
x = [1346139060.0,
   1346139098.0,
   1346139113.0,
   1346139053.0,
   1346139234.0,
   1346139006.0,
   1346139209.0,
   1346139147.0,
   1346138998.0,
   1346139206.0,
   1346139224.0,
   1346139174.0,
   1346139152.0,
   1346139230.0,
   1346139181.0,
   1346139159.0,
   1346139314.0]
(max(x) - min(x)) / float(len(x))

18.58823529411765

In [40]:
topUsers = avgRatingFreq.top(10, key=lambda x: x[1])

In [41]:
topUsers

[('40407', 51121853.75),
 ('241087', 40917744.72727273),
 ('54838', 36601016.0),
 ('248290', 33999095.666666664),
 ('39601', 33138222.0),
 ('155302', 33013341.666666668),
 ('117995', 29406107.25),
 ('183383', 29210786.666666668),
 ('121552', 26685917.875),
 ('74936', 26309319.0)]

In [4]:
sc.stop()